In [227]:
import os  # when loading file paths
import pandas as pd  # for lookup in annotation file
import spacy  # for tokenizer
import torch
from torch.nn.utils.rnn import pad_sequence  # pad batch
from torch.utils.data import DataLoader, Dataset
from PIL import Image  # Load img
import torchvision.transforms as transforms
from torchtext.legacy.data import Field, TabularDataset, BucketIterator
from sklearn.model_selection import train_test_split
import string
import numpy as np
from torch import nn
from torch.autograd import Variable
from torchvision import datasets, models, transforms
import torchvision
import torch.optim as optim
import matplotlib.pyplot as plt
import nltk
import torchvision.transforms as tfs
import torch.nn.functional as F
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import itertools
from sklearn.metrics import accuracy_score
from sklearn import  metrics

In [228]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [229]:
def myTokenize(text):
    return nltk.word_tokenize(text)

In [230]:
lineToInd = {'Cause-Effect(e1,e2)':0, 'Cause-Effect(e2,e1)':0,
              'Instrument-Agency(e1,e2)':1,'Instrument-Agency(e2,e1)':1,
              'Product-Producer(e1,e2)':2,'Product-Producer(e2,e1)':2,
             'Content-Container(e1,e2)':3, 'Content-Container(e2,e1)':3,
             'Entity-Origin(e1,e2)':4, 'Entity-Origin(e2,e1)':4,
             'Entity-Destination(e1,e2)':5, 'Entity-Destination(e2,e1)':5,
             'Component-Whole(e1,e2)':6,'Component-Whole(e2,e1)':6,
             'Member-Collection(e1,e2)':7,'Member-Collection(e2,e1)':7,
             'Message-Topic(e1,e2)':8,'Message-Topic(e2,e1)':8,
             'Entity-Destination(e1,e2)':5 ,'Other':9}

In [231]:
#http://analyticmunch.weebly.com/semeval-2010-task-8.html
import re
import csv

def read_line_by_line(file_txt):
    all_lines = []
    with open(file_txt, 'r') as file:
        temp_lines = file.readlines()
        for line in temp_lines:
            line = line.strip()
            if line:
                all_lines.append(line)
    return all_lines

def load_extract(file_txt):
    al_train = read_line_by_line(file_txt)
    train_raw = []
    i=0
    for line1 in al_train:
        train_raw.append(line1.split('|'))
        i = i+1
    #print train_raw
    odd = []
    even = []
#Removing the comment line, for the hackathon purpose we remove the comment line from text file
    for line in train_raw:
        com = re.compile(r"Comment:.*") 
        new = filter(com.match,line)
        if line == new:
            train_raw.remove(line)
        else:
            continue
    e1e2 = []
    for odd_line in train_raw:
        odd_string = ''.join(odd_line)
        oddmatch =re.sub('\t','',odd_string)
        k=oddmatch.lstrip('0123456789.-')
        asub = re.sub('<e1>', ' e11 ', k)
        asub = asub.replace('</e1>', ' e12 ')
        asub = asub.replace('<e2>', ' e21 ')
        asub = asub.replace('</e2>', ' e22 ')
        asub = odd_line2= asub.replace('"', '')
        e1e2.append(odd_line2)
    return e1e2


In [232]:
#text = load_extract('/content/drive/MyDrive/Hw3/TRAIN_FILE.TXT')
text = load_extract('/content/drive/MyDrive/Hw3/TEST_FILE_FULL.TXT')
sentence = []
comment = []
relation = []
for cnt,line in enumerate(text):
  if cnt%3==0:
    sentence.append(line)
  elif cnt%3==1:
    relation.append(lineToInd[line])
  else:
    comment.append(line)
    
dataFrame = pd.DataFrame(list(zip(sentence, relation)),
               columns =['sentence', 'relation'])

In [233]:
for punc in string.punctuation+'\n':
      dataFrame["sentence"] = dataFrame.sentence.str.replace(punc, "")

In [234]:
entities = ['e11', 'e12', 'e21', 'e22']
attentions = np.zeros([len(dataFrame), 4])
for i in range(len(dataFrame)):
  index = []
  for cnt, entity in enumerate(entities):
    cnt = cnt
    amirStr = dataFrame['sentence'].iloc[i]
    index.append(myTokenize(amirStr).index(entity)-cnt)
  attentions[i,:] = np.asarray(index)

In [235]:
attentions = attentions.astype(int)

In [236]:
Myentities = ['e11', 'e12', 'e21', 'e22']
for entity in Myentities:
  dataFrame['sentence'] = dataFrame.sentence.str.replace(entity, "")

In [237]:
dataFrame['e11'] = attentions[:,0]
dataFrame['e12'] = attentions[:,1]
dataFrame['e21'] = attentions[:,2]
dataFrame['e22'] = attentions[:,3]

In [238]:
dataFrame

,sentence,relation,e11,e12,e21,e22
0,The most common audits were about waste ...,8,3,4,6,7
1,The company fabricates plastic chairs,2,1,2,4,5
2,The school master teaches the lesson with ...,1,2,3,8,9
3,The suspect dumped the dead body into a lo...,5,5,6,9,10
4,Avian influenza is an infectious disease o...,0,1,2,16,17
...,...,...,...,...,...,...
2712,After seating all the idols which itself takes...,1,11,12,18,19
2713,The minister attributed the slow production of...,2,8,9,12,13
2714,The umbrella frame is provided with a ...,6,1,2,2,3
2715,Manos The Hands of Fate is a lowbudget horror ...,2,9,10,15,16


In [239]:
# dataFrame.to_csv('train_file.csv', index=False)
dataFrame.to_csv('test_file.csv', index=False)